In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

# === 📥 LOAD DATA ===
df = pd.read_csv("C:/Users/mloai/Event-Aware Sales Forecasting with Time Series Models/processed_sales.csv")
df['CAL_DATE'] = pd.to_datetime(df['CAL_DATE'], errors='coerce')
df['HIJRI_DATE'] = df['HIJRI_DATE'].astype(str)
df['Hijri_Month'] = df['HIJRI_DATE'].str.split('-').str[1].astype(int)
df = df.dropna(subset=['CAL_DATE'])
print(df['HIJRI_DATE'].dtype)
print(df['HIJRI_DATE'].head())

object
0    1443-05-28
1    1443-05-28
2    1443-05-28
3    1443-05-28
4    1443-05-28
Name: HIJRI_DATE, dtype: object


In [2]:
df['CAL_DATE'] = pd.to_datetime(df['CAL_DATE'], errors='coerce')
df = df.dropna(subset=['CAL_DATE'])  # drop invalid dates

In [3]:
# === 📅 GREGORIAN FEATURES (Saudi-Aligned) ===

df['Day'] = df['CAL_DATE'].dt.day
df['Month'] = df['CAL_DATE'].dt.month
df['Year'] = df['CAL_DATE'].dt.year
df['Quarter'] = df['CAL_DATE'].dt.quarter
df['DayOfYear'] = df['CAL_DATE'].dt.dayofyear
df['Week'] = df['CAL_DATE'].dt.isocalendar().week.astype(int)

# Saudi-localized weekday (Sunday=0, Saturday=6)
df['Weekday'] = (df['CAL_DATE'].dt.weekday + 1) % 7

# Weekend: Friday (5), Saturday (6)
df['Is_Weekend'] = df['Weekday'].isin([5, 6]).astype(int)

# Week start: Sunday (0)
df['Is_Week_Start'] = (df['Weekday'] == 0).astype(int)

df['Is_Month_Start'] = df['CAL_DATE'].dt.is_month_start.astype(int)
df['Is_Month_End'] = df['CAL_DATE'].dt.is_month_end.astype(int)

df['Days_Since_Start'] = (df['CAL_DATE'] - df['CAL_DATE'].min()).dt.days

# Initialize the column with 0
df['Is_Pre_Ramadan'] = 0



# Extract Hijri month from a "YYYY-MM-DD" string
df['Hijri_Month'] = df['HIJRI_DATE'].str.split('-').str[1].astype(int)

In [4]:
# === 🎯 RAW EVENT MAPPING ===
event_dict = {
    # Ramadan
    "Ramadan Wave 1": "Ramadan_Campaign",
    "Ramadan Wave 2": "Ramadan_Campaign",
    "Ramadan - Phase I": "Ramadan_Campaign",
    "Ramadan - Phase Ii": "Ramadan_Campaign",
    "Ramadan Wave 1 & Iphone 10%": "Ramadan_Campaign",
    "Ramadan Wave 1 & Computer 10%": "Ramadan_Campaign",
    "Ramadan Wave 2& Mother Day 10% Discount": "Ramadan_Campaign",
    
    # Eid
    "Eid": "Eid",
    "Eid Campaign": "Eid",
    "Adha": "Eid",

    # Mega Sale
    "Mega Sale": "Mega_Sale",
    "Mega Sale Teaser": "Mega_Sale",
    "White Friday": "Mega_Sale",
    "Pre-Mega Sale Bundles": "Mega_Sale",
    "Daily Deals": "Mega_Sale",
    "Clearance Event": "Mega_Sale",
    "Weekend Offers": "Mega_Sale",

    # Product Launch
    "Iphone 13 Pre Order": "Product_Launch",
    "Iphone 13 Launch": "Product_Launch",
    "Iphone Pre-Order": "Product_Launch",
    "Samsung S22 Pre-Order": "Product_Launch",
    "S22 Launch": "Product_Launch",
    "S 23 Teaser": "Product_Launch",

    # Campaign Driven
    "Digital Mania": "Campaign_Driven",
    "Digital Mania & Samsung S23 Pre-Order": "Campaign_Driven",
    "10% Additional Discount Ce": "Campaign_Driven",
    "Fd 10% Off": "Campaign_Driven",
    "Online Special Day": "Campaign_Driven",
    "Tamara Start Online": "Campaign_Driven",
    "Rajhi Bank": "Campaign_Driven",
    "Riyad Bank Promo": "Campaign_Driven",
    "Al-Rajhi Bank Promo": "Campaign_Driven",
    "Aljazira Bank Offer": "Campaign_Driven",
    "Sabb": "Campaign_Driven",

    # Back to School
    "Bts": "Back_To_School",
    "Mini Bts": "Back_To_School",
    "Bts Phase 2": "Back_To_School",

    # Seasonal
    "Summer": "Seasonal_Event",
    "Winter": "Seasonal_Event",
    "Spring": "Seasonal_Event",
    "Summer Extend": "Seasonal_Event",
    "Winter Theme": "Seasonal_Event",
    "Spring Theme": "Seasonal_Event",
    "Vacation": "Seasonal_Event",

    # National
    "Founding Day": "National_Event",
    "Father Day": "National_Event",
    "Single Day": "National_Event",
    "Anniversary": "National_Event",
    "Jood Day": "National_Event",

    # Rare
    "World Cup": "Rare_Event",
    "Euro": "Rare_Event",
    "Nd": "Rare_Event",

    # Calendar
    "End Of Year": "Calendar_Promo",
    "Pay Week": "Calendar_Promo",
    "Weekend Theme": "Calendar_Promo"
}


In [5]:
# Standardize Main Event column (remove NaNs and strip whitespace)
df['Main Event'] = df['Main Event'].fillna('').str.strip()

# Define lists of event names per season
summer_events = ['Summer', 'Summer Extend']
winter_events = ['Winter', 'Winter Theme']
spring_events = ['Spring', 'Spring Theme']
Vacation_events = ['Vacation']


# Create binary features based on actual events triggered
df['Is_Summer'] = df['Main Event'].isin(summer_events).astype(int)
df['Is_Winter'] = df['Main Event'].isin(winter_events).astype(int)
df['Is_Spring'] = df['Main Event'].isin(spring_events).astype(int)
df['Is_Vacation'] = df['Main Event'].isin(Vacation_events).astype(int)

In [6]:
# Make sure CAL_DATE is datetime
df['CAL_DATE'] = pd.to_datetime(df['CAL_DATE'], errors='coerce')

# Standardize event names
df['Main Event'] = df['Main Event'].fillna('').str.strip()

# Define seasonal events
season_map = {
    'Summer': ['Summer', 'Summer Extend'],
    'Winter': ['Winter', 'Winter Theme'],
    'Spring': ['Spring', 'Spring Theme'],
    'Vacation': ['Vacation']
}

# Create summary list
season_ranges = []

# Loop through each season
for season, event_names in season_map.items():
    # Filter rows for the season
    season_df = df[df['Main Event'].isin(event_names)].copy()
    season_df['Year'] = season_df['CAL_DATE'].dt.year

    # Group by year and get min/max CAL_DATE
    grouped = season_df.groupby('Year')['CAL_DATE'].agg(['min', 'max']).reset_index()
    grouped['Season'] = season
    season_ranges.append(grouped)

# Combine all seasons
season_timeline = pd.concat(season_ranges).sort_values(['Season', 'Year'])

# Reset index and show
season_timeline = season_timeline[['Season', 'Year', 'min', 'max']].reset_index(drop=True)
season_timeline.columns = ['Season', 'Year', 'Start_Date', 'End_Date']

# Display the result
print(season_timeline)

     Season  Year Start_Date   End_Date
0    Spring  2024 2024-04-25 2024-05-11
1    Spring  2025 2025-04-10 2025-04-19
2    Summer  2022 2022-06-23 2022-07-16
3    Summer  2023 2023-06-15 2023-07-19
4  Vacation  2022 2022-07-28 2022-08-13
5  Vacation  2023 2023-07-20 2023-08-05
6  Vacation  2024 2024-07-15 2024-07-31
7    Winter  2022 2022-01-27 2022-02-05
8    Winter  2024 2024-01-07 2024-01-24
9    Winter  2025 2025-01-12 2025-01-22


In [7]:
def map_event(raw):
    if pd.isna(raw) or raw.strip().lower() in ['none', '']:
        return 'Regular'
    return event_dict.get(raw.strip(), 'Regular')

df['event_category'] = df['Main Event'].apply(map_event)

# === 🕋 FIX RAMADAN GAPS ===
df.loc[(df['Hijri_Month'] == 9) & (df['event_category'] == 'Regular'), 'event_category'] = 'Ramadan_Campaign'

# === 🧠 MERGE TO 8 MODELING CATEGORIES ===
weak = ['Product_Launch', 'Rare_Event']
df['event_category_merged'] = df['event_category'].apply(
    lambda x: 'Other_Event' if x in weak else x
)

# Ensure it's a string category (not bool or object with mixed types)
df['event_category_merged'] = df['event_category_merged'].astype(str)

# One-hot encode directly to 0/1
df = pd.get_dummies(df, columns=['event_category_merged'], prefix='event', dtype='uint8')

In [8]:
# Set to 1 when it's a Ramadan campaign but not during actual Hijri month 9
df.loc[
    (df['event_category'] == 'Regular') & (df['Is_Ramadan'] == 0),
    'Is_Pre_Ramadan'
] = 1

In [9]:
# Create the column and initialize to 0
df['Is_regular'] = 0

# Set to 1 where event_category is 'Regular'
df.loc[df['event_category'] == 'Regular', 'Is_regular'] = 1

In [10]:
# Create the column and initialize to 0
df['Is_MegaSale'] = 0

# Set to 1 where event_category is 'Regular'
df.loc[df['event_category'] == 'Mega_Sale', 'Is_MegaSale'] = 1


In [11]:
# Global sort for stable order
df = df.sort_values(by=['Category_Channel', 'CAL_DATE']).reset_index(drop=True)
# Apply lag/rolling features inside each CATEGORY_CODE group
processed_groups = []

for cat_code, group in df.groupby('Category_Channel'):
    group = group.sort_values(by='CAL_DATE')  # Local sort again just in case
  
    # Create lag and rolling features
    group['sales_lag_1'] = group['LOG_SALES'].shift(1)
    group['sales_lag_14'] = group['LOG_SALES'].shift(14)

    
    group['sales_roll_mean_7'] = group['LOG_SALES'].rolling(7, min_periods=1).mean()
    group['sales_roll_mean_14'] = group['LOG_SALES'].rolling(14, min_periods=1).mean()

    
    group['sales_roll_std_7'] = group['LOG_SALES'].rolling(7, min_periods=1).std()
    group['sales_roll_std_14'] = group['LOG_SALES'].rolling(14, min_periods=1).std()

    processed_groups.append(group)
  
# Combine and export
df_final = pd.concat(processed_groups).reset_index(drop=True)
df_final = df_final.dropna(subset=[
    'sales_lag_1' ,'sales_lag_14',
    'sales_roll_mean_7', 'sales_roll_mean_14', 
    'sales_roll_std_7', 'sales_roll_std_14', 
])

In [13]:
# === 💾 SAVE ===
df.to_csv("sales_with_features.csv", index=False)
df_final.to_csv("sales_with_features1.csv", index=False)
print("✅ Final version with fixed Hijri + exact event mapping complete")

✅ Final version with fixed Hijri + exact event mapping complete
